This notebook takes the 1h price cleansed data from and enriches it with other context  

In [0]:
import pyspark.sql.functions as sf

In [0]:
# Read data from 'runescape.02_silver.1h_prices_cleansed'
df_1hour_cleansed = spark.read.table("runescape.02_silver.1h_prices_cleansed")

Should the data be limited to the last time only?  
This could improve performance?

In [0]:
# create df with only the latest 1h_prices
# get latest time
df_latest_time = df_1hour_cleansed.select(sf.max("time").alias("time"))
# join both df's to get a df with only the latest hourly price data
df_1hour_cleansed_latest = df_1hour_cleansed.join(df_latest_time, "time")

In [0]:
# write latest data to table 'runescape.02_silver.1h_prices_latest_enriched'

targetDF = DeltaTable.forName(spark, "runescape.02_silver.latest_prices_cleansed")
dfUpdates = df_latest_prices

targetDF.alias("t") .\
  merge(
    source = dfUpdates.alias("s"),
    condition = "t.id = s.id AND t.time = s.time AND \
         t.highorlow = s.highorlow") .\
  whenNotMatchedInsertAll() .\
  execute()

In [0]:
df_1hour_cleansed.display()

In [0]:
df_agg = df_1hour_cleansed.groupBy("time").count()
df_agg.display()